In [2]:
cd /home/

/home


In [22]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import os, sys
import h5py
import json

from sklearn.metrics import confusion_matrix
from core.plots import plot_cm
from core.phased import PhasedLSTM
from core.data import load_records
from core.losses import custom_bce
from core.metrics import custom_acc

In [19]:
exp_path = './runs/forced/fold_0/phased/'
data_path = './data/records/forced/fold_0/'

In [23]:
model = tf.keras.models.load_model(
        os.path.join(exp_path, 'model.h5'),
        custom_objects={'PhasedLSTM': PhasedLSTM, 
                        'custom_bce':custom_bce,
                        'custom_acc':custom_acc}
    )

In [24]:
metadata = pd.read_csv(os.path.join(data_path, 'test_samples.csv'))
num_classes = len(metadata['alerceclass'].unique())
test_batches = load_records(os.path.join(data_path, 'test'),
                             16,
                             max_obs=200,
                             num_classes=num_classes,
                             sampling=False,
                             shuffle=False)


In [34]:
def custom_bce(y_true, y_pred, sample_weight=None):
    mask   = y_pred[...,-1]
    y_pred = y_pred[...,:-1]

    num_steps = tf.shape(y_pred)[1]

    y_true = tf.expand_dims(y_true, 1)
    y_true = tf.tile(y_true, [1, num_steps, 1])

    losses = tf.nn.softmax_cross_entropy_with_logits(y_true, y_pred)
    losses = tf.multiply(losses, mask)
    losses = tf.reduce_sum(losses, 1)
    losses = tf.divide(losses, tf.reduce_sum(mask))
    return tf.reduce_mean(losses)

In [35]:
for batch, y in test_batches:
    y_pred = model(batch)
    custom_bce(y, y_pred)
    break

tf.Tensor(
[[0.8810056  0.68829393 0.5618404  ... 0.04955821 0.04979631 0.05091546]
 [1.0118897  0.8520433  0.74179184 ... 0.0548726  0.05431003 0.05369619]
 [0.93474627 0.7612856  0.61366606 ... 0.1430687  0.14316066 0.14305869]
 ...
 [1.0088143  0.9023385  0.77803475 ... 0.03742049 0.03750362 0.03765081]
 [1.002216   0.8850013  0.7782019  ... 0.02976951 0.02880991 0.02767164]
 [1.0678198  1.0443408  1.0112319  ... 0.         0.         0.        ]], shape=(16, 200), dtype=float32)
(16,)


2021-11-22 21:22:36.934439: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [33]:
# fig, axes = plt.subplots(1,2, figsize=(16,5), sharey=True, gridspec_kw={'hspace': 0.1, 'wspace': 0.01}, dpi=300)

# plot_cm(cm_0, axes[0], title=metrics_0['title'], class_names=cls_names_0, fontsize=10)
# plot_cm(cm_1, axes[1], title=metrics_1['title'], class_names=cls_names_0, fontsize=10)
# # plot_cm(cm_2, axes[2], title=metrics_2['title'], class_names=cls_names_0, fontsize=10)

In [13]:
# import pandas as pd

# print('Model & F1 & Recall & Precision & Accuracy \\\\\hline')
# for dictonary in [metrics_0,metrics_1]:
#     print('{} & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\\hline '.format(dictonary['title'],
#                                                           dictonary['f1'],
#                                                           dictonary['recall'],
#                                                           dictonary['precision'],
#                                                           dictonary['accuracy']))